 # Opis zadania
 W tym notatniku stworzymy system analityczno-raportowy, mający wspomagać przyszłe decyzje biznesowe. Dzięki przeprowadzonej wcześniej analizie eksploracyjnej danych, w trakcie spotkania prezentującego wykonaną pracę postanowiono w pierwszym kroku stworzyć następujące wizualizacje danych:
 - Wyświetlenie TOP 10 tras samolotów pod względem najniższego odsetka opóźnień w formie tabeli,
 - Wyświetlenie porównania roku 2019 oraz 2020 w formie wykresu słupkowego:
     - miesiąc do miesiąca,
     - dzień tygodnia do dnia tygodnia,
 - Wyświetlenie danych dzień po dniu w formie szeregu czasowego.

Raport powinien składać się z trzech stron - `TOP report`, `Comparision` i `Day by day reliability`.

Pracując na podstawie wcześniej zdefiniowanych widoków, nie musimy się tutaj martwić o przetwarzanie oraz procesowanie danych. Co więcej, wszystkie niuanse techniczne w postaci liczby wierszy, wykonywanych złączeń, filtracji są przeniesione na bazę danych. Z technicznego punktu widzenia, pomiędzy serwisem a bazą danych dochodzi do przesyłania mniejszej ilości danych.

Dopuszczamy jednak pewne aktualizacje co do ich struktury przykładowo poprzez _pivotowanie_, czyli obranie i doprowadzanie do postaci tej znanej z tabel przestawnych.

> **Uwaga:**  
> Przy pracy nad poprawkami w dashboardzie, pamiętaj, że aby odświeżyć stronę po wprowadzonych zmianach, należy **całość** uruchomić ponownie.

W tej części projektu końcowego wcielasz się ponownie w rolę BI Developera, który ma za zadanie stworzyć dashboard zgodny z wytycznymi biznesowymi dostarczonymi przez klienta. Pamiętaj, że osoba na tym stanowisku często ma kontakt z biznesem więc musi umieć przekazać informację o danych, które zawarła na dashboardzie by te zaangażowały odbiorców.


 Tutaj zaimportuj potrzebne biblioteki

In [94]:

import psycopg2 as pg
import pandas as pd
import plotly.express as px
import calendar
import dash
import threading


from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [95]:
username = 'postgres'
password = 'Wawa2023'

host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [96]:
con = pg.connect(database=database, user=username, host=host, password=password, port=port)
cursor = con.cursor()

 # Stworzenie tabeli dla `TOP 10 routes`
 W tym miejscu przygotujemy tabelkę oraz komponent, który zostanie następnie umieszczony w raporcie. Tabela powinna się składać z następujących kolumn:
 - Origin,
 - Destination,
 - Year,
 - Reliability,
 - Rank.

Wartości procentowe zaś powinny być postaci np. 13.87%.

Z tego względu konieczna będzie zmiana nazw kolumn dla wyników raportowania.

Wskazówki:
 - `Python - analiza danych > Dzień 5 - Pandas > Obróbka danych - część 2 > apply`,
 - [How to format percentage in python](https://stackoverflow.com/questions/5306756/how-to-print-a-percentage-value-in-python).

 Tutaj zaczytaj dane do ramki danych `top_routes_df` z widoku `reporting.top_reliability_roads`
 > Jeśli pojawi się komunikat `UserWarning: pandas only support SQLAlchemy`, z naszego punktu widzenia, możemy go śmiało zignorować.

In [97]:
top_routes_df = pd.read_sql("SELECT * FROM reporting.top_reliability_roads", con)
top_routes_df

C:\Users\Home\AppData\Local\Temp\ipykernel_15300\2115013962.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,origin_airport_id,origin_airport_name,dest_airport_id,dest_airport_name,year,cnt,reliability,nb
0,12173,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",13830,"HONOLULU INTERNATIONAL AIRPORT, HI US",2019,10735,0.25,1
1,13830,"HONOLULU INTERNATIONAL AIRPORT, HI US",12173,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,10738,0.27,2
2,12478,"LAGUARDIA AIRPORT, NY US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,12804,0.27,2
3,14771,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,14614,0.33,3
4,10721,"BOSTON, MA US",12953,"LAGUARDIA AIRPORT, NY US",2019,10303,0.34,4
5,12953,"LAGUARDIA AIRPORT, NY US",10721,"BOSTON, MA US",2019,10309,0.34,4
6,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",14771,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,14630,0.35,5
7,12953,"LAGUARDIA AIRPORT, NY US",13930,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,14699,0.35,5
8,12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,11718,0.36,6
9,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,11722,0.37,7


 Tutaj dokonaj aktualizacji formatu liczbowego dla kolumny `reliability`

In [98]:
top_routes_df['reliability'] = top_routes_df['reliability'].apply(lambda x: f"{x*100:.2f}%")


 Tutaj odpowiednio dokonaj zmian nazewnictwa kolumn

In [99]:
top_routes_df.rename(columns={'origin_airport_name':'Origin', 'dest_airport_name':'Destination', 'year':'Year', 'reliability':'Reliability', 'nb':'Rank'}, inplace=True)
top_routes_df_final = top_routes_df[['Origin', 'Destination', 'Year', 'Reliability', 'Rank']]
top_routes_df_final

,Origin,Destination,Year,Reliability,Rank
0,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","HONOLULU INTERNATIONAL AIRPORT, HI US",2019,25.00%,1
1,"HONOLULU INTERNATIONAL AIRPORT, HI US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,27.00%,2
2,"LAGUARDIA AIRPORT, NY US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,27.00%,2
3,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,33.00%,3
4,"BOSTON, MA US","LAGUARDIA AIRPORT, NY US",2019,34.00%,4
5,"LAGUARDIA AIRPORT, NY US","BOSTON, MA US",2019,34.00%,4
6,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,35.00%,5
7,"LAGUARDIA AIRPORT, NY US","CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",2019,35.00%,5
8,"MCCARRAN INTERNATIONAL AIRPORT, NV US","LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,36.00%,6
9,"LOS ANGELES INTERNATIONAL AIRPORT, CA US","MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,37.00%,7


 # Porównanie rok do roku (YoY) 2019 vs. 2020
 W tym miejscu stworzymy wykres oraz komponent, który zostanie następnie umieszczony w raporcie. Wykres powinien przedstawiać porównanie lat 2019 oraz 2020 po miesiącach względem zmiennej `reliability`.

 > Na wykresie chcemy również przedstawić dane, które nie mają porównania tj. od kwietnia do grudnia 2019.

W tym celu wykonamy poniższe zadania:
 - Przekształcimy tabelkę do formy tabeli przestawnej
 - Stworzymy wykres słupkowy porównujący dane rok do roku
 - Opakujemy stworzony wykres w komponent `Dash`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 1 > Budowanie wykresów plotly`,
 - `Python - analiza danych > Dzień 6 - Pandas c.d > Pivot`.

 Tutaj pobierz z widoku `reporting.year_to_year_comparision` dane do ramki `yoy_comparision_df`

In [100]:
yoy_comparison_df = pd.read_sql("SELECT * FROM reporting.year_to_year_comparision", con)
yoy_comparison_df['month'] = yoy_comparison_df['month'].apply(lambda x: calendar.month_name[x])
yoy_comparison_df

C:\Users\Home\AppData\Local\Temp\ipykernel_15300\2484804727.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,month,flights_amount,reliability
0,2019,January,89455,0.34
1,2019,February,81574,0.39
2,2019,March,93862,0.35
3,2019,April,91434,0.34
4,2019,May,96097,0.38
5,2019,June,94948,0.43
6,2019,July,98042,0.39
7,2019,August,100496,0.38
8,2019,September,92090,0.30
9,2019,October,96566,0.33


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `yoy_comparision_to_plot_df`

In [101]:
yoy_comparison_to_plot = pd.pivot(yoy_comparison_df, index='month', columns='year', values='reliability')
months_order = list(calendar.month_name)[1:]
yoy_comparison_to_plot = yoy_comparison_to_plot.reindex(months_order)
yoy_comparison_to_plot

year,2019,2020
month,,
January,0.34,0.28
February,0.39,0.28
March,0.35,0.16
April,0.34,NaN
May,0.38,NaN
June,0.43,NaN
July,0.39,NaN
August,0.38,NaN
September,0.30,NaN


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `yoy_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [102]:

yoy_comparision_fig = px.bar(yoy_comparison_to_plot,
                            x=yoy_comparison_to_plot.index,
                            y=yoy_comparison_to_plot.columns,
                            barmode='group',
                            color='year',
                            template="plotly_white",
                            title='Year to Year Reliability Comparison'
                            )                            
yoy_comparision_fig.update_layout(showlegend=True, 
                                  title_x=0.5, 
                                  title_font=dict(size=25),
                                  xaxis_title="Month",
                                  yaxis_title="Reliability")
yoy_comparision_fig.show()

 # Porównanie dzień tygodnia do dnia tygodnia (WKoWK) 2019 vs. 2020
 Wykonaj analogiczne do poprzedniego kroku, używając jednak w tym momencie danych z widoku `reporting.day_to_day_comparision`

 Tutaj pobierz z widoku `reporting.day_to_day_comparision` dane do ramki `day_to_day_comparision_df`

In [103]:
day_to_day_comparision_df = pd.read_sql("SELECT * FROM reporting.day_to_day_comparision", con)
day_to_day_comparision_df['day_of_week'] = day_to_day_comparision_df['day_of_week'].apply(lambda x: calendar.day_name[x-1])
day_to_day_comparision_df

C:\Users\Home\AppData\Local\Temp\ipykernel_15300\4200275071.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,day_of_week,flights_amount
0,2019,Monday,167761
1,2019,Tuesday,165478
2,2019,Wednesday,163889
3,2019,Thursday,166154
4,2019,Friday,167397
5,2019,Saturday,129485
6,2019,Sunday,155663
7,2020,Monday,40983
8,2020,Tuesday,39807
9,2020,Wednesday,39581


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `day_to_day_comparision_to_plot_df`

In [104]:
day_to_day_comparision_to_plot_df = pd.pivot(day_to_day_comparision_df, index='day_of_week', columns='year', values='flights_amount')
days_order = list(calendar.day_name)
day_to_day_comparision_to_plot_df = day_to_day_comparision_to_plot_df.reindex(days_order)
day_to_day_comparision_to_plot_df

year,2019,2020
day_of_week,,
Monday,167761,40983
Tuesday,165478,39807
Wednesday,163889,39581
Thursday,166154,41086
Friday,167397,41058
Saturday,129485,30545
Sunday,155663,37233


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `day_to_day_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [105]:
day_to_day_comparision_fig = px.bar(day_to_day_comparision_to_plot_df,
                            x=day_to_day_comparision_to_plot_df.index,
                            y=day_to_day_comparision_to_plot_df.columns,
                            barmode='group',
                            color='year',
                            template="plotly_white",
                            title='Day to Day Flights Comparison'
                            )                            
day_to_day_comparision_fig.update_layout(showlegend=True, 
                                  title_x=0.5, 
                                  title_font=dict(size=25),
                                  xaxis_title="Day of week",
                                  yaxis_title="Flights amount")
day_to_day_comparision_fig.show()

 # Stworzenie szeregu czasowego
 W tym miejscu stworzymy wykres w formie szeregu czasowego, który zostanie umieszcony w raporcie. Wykres powinien przedstawiać dane w formie szeregu czasowego (dzień po dniu) dla lat 2019 oraz 2020. Dla możliwości identyfikacji poszczególnych lat, zostaliśmy dodatkowo poproszeni o nadanie innego koloru dla roku 2019 oraz 2020.

 > Pamiętaj o stworzeniu wykresu zgodnie z dobrymi praktykami.

 W tym miejscu pobierz dane do ramki `day_by_day_reliability_df`, z widoku `reporting.day_by_day_reliability`.

In [106]:
day_by_day_reliability_df = pd.read_sql("SELECT * FROM reporting.day_by_day_reliability", con)
day_by_day_reliability_df

C:\Users\Home\AppData\Local\Temp\ipykernel_15300\1639168378.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,date,reliability
0,2019-01-01,0.40
1,2019-01-02,0.44
2,2019-01-03,0.35
3,2019-01-04,0.29
4,2019-01-05,0.37
...,...,...
451,2020-03-27,0.05
452,2020-03-28,0.07
453,2020-03-29,0.05
454,2020-03-30,0.05


 Tutaj stwórz wykres liniowy na podstawie pobranych danych. Wynik zapisz do zmiennej `day_by_day_reliability_fig`.

In [107]:
day_by_day_reliability_fig = px.line(day_by_day_reliability_df,
                            x='date',
                            y='reliability',
                            title='Day by day reliability',                            
                            labels={'date': 'Date', 'reliability': 'Reliability'},
                            template="plotly_white"
)
day_by_day_reliability_fig.update_layout(title_x=0.5, 
                                         title_font=dict(size=25),)
day_by_day_reliability_fig.show()

 # Stworzenie layoutów poszczególnych stron

 ## Stworzenie layout dla widoku `TOP 10 routes`
 W tym momencie przekonwertujemy zmienną `top_routes_df` w postać `Dash.DataTable`. Dodatkowo, dla czytelności, dodamy nagłówek opisujący co tabela przedstawia.

 Nagłówek wystylizuj używając poniższych wymagań:
 - tag: `H3`,
 - czcionka: `verdana`,
 - kolor: '#4444`,
 - tekst do wyświetlenia: `TOP 10 reliability routes in 2019 and 2020`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 3 > Dash - datatable`,
 - Dokmentacja metody `Pandas` - [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html).

 Tutaj przygotuj komponent `Dash` `DataTable`, zapisując wynik do zmiennej `top_routes_table`

In [108]:
top_routes_df = pd.read_sql("SELECT * FROM reporting.top_reliability_roads", con)

C:\Users\Home\AppData\Local\Temp\ipykernel_15300\2965344040.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [109]:
top_routes_table = dash_table.DataTable(id='top_routes',data=top_routes_df.to_dict('records'), columns=[{'name': i, 'id': i} for i in top_routes_df.columns])
top_routes_table

DataTable(data=[{'origin_airport_id': 12173, 'origin_airport_name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'dest_airport_id': 13830, 'dest_airport_name': 'HONOLULU INTERNATIONAL AIRPORT, HI US', 'year': 2019, 'cnt': 10735, 'reliability': 0.25, 'nb': 1}, {'origin_airport_id': 13830, 'origin_airport_name': 'HONOLULU INTERNATIONAL AIRPORT, HI US', 'dest_airport_id': 12173, 'dest_airport_name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'year': 2019, 'cnt': 10738, 'reliability': 0.27, 'nb': 2}, {'origin_airport_id': 12478, 'origin_airport_name': 'LAGUARDIA AIRPORT, NY US', 'dest_airport_id': 12892, 'dest_airport_name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'year': 2019, 'cnt': 12804, 'reliability': 0.27, 'nb': 2}, {'origin_airport_id': 14771, 'origin_airport_name': 'SAN FRANCISCO INTERNATIONAL AIRPORT, CA US', 'dest_airport_id': 12892, 'dest_airport_name': 'LOS ANGELES INTERNATIONAL AIRPORT, CA US', 'year': 2019, 'cnt': 14614, 'reliability': 0.33, 'nb': 3}, {'origin_airport_id'

 Tutaj przygotuj nagłówek dla tabeli, zapisz go pod nazwą `top_routes_page_title`

In [110]:
top_routes_page_title = html.H3('TOP 10 reliability routes in 2019 and 2020', style={'fontFamily': 'Verdana', 'color': '#4444'})

 Tutaj przygotuj layout dla widoku `TOP 10 routes`. Konfigurację zapisz do zmiennej `top_routes_layout`

In [111]:
top_routes_layout = html.Div([top_routes_page_title, 
                              top_routes_table])

 ## Stworzenie layout dla widoku `Comparision`
 W tym momencie opakujemy zmienne `yoy_comparision_fig` oraz `day_to_day_comparision_fig` do komponentów `Dash`, a następnie stworzymy odpowiedni `layout` dla strony `comparision`. Wyniki zapiszemy do zmiennej `comparision_layout`.

 Tutaj opakuj `yoy_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `yoy_comparision_component`.

In [112]:
yoy_comparision_component = html.Div([
        dcc.Graph(
            id='yoy_comparision_component',
            figure=yoy_comparision_fig
        )
])

 Tutaj opakuj `day_to_day_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `day_to_day_comparision_component`.

In [113]:
day_to_day_comparision_component = html.Div([
        dcc.Graph(
            id='day_to_day_comparision_component',
            figure=day_to_day_comparision_fig
        )
])

 Tutaj stwórz odpowiedni `layout` dla widoku `comparision`. Wynik zapisz do zmiennej `comparision_layout`.

In [114]:
comparision_layout = html.Div([
    html.Div([
        yoy_comparision_component
     ]),
    html.Div([
        day_to_day_comparision_component
    ])
])

 Tutaj stwórz odpowiedni `layout` dla widoku `day by day`. Wynik zapisz do zmiennej `day_by_day_layout`.

In [115]:
day_by_day_layout = html.Div([
        dcc.Graph(
            id='day_by_day_relibility',
            figure=day_by_day_reliability_fig
        )
])

 # Konfigurowanie aplikacji
 Posiadamy już wszystkie niezbędne komponenty do stworzenia oraz uruchomienia aplikacji. Naszym zadaniem w tym miejscu będzie odpowiednie jej skonfigurowanie, aby obsługiwała więcej niż jedną stronę.

 Chcemy, aby możliwe było poruszanie się pomiędzy poniższymi stronami:
 - `TOP report` - strona domyślna,
 - `Comparision` - strona dostępna pod adresem: `comparision_reporting`,
 - `Day by day reporting` - strona dostępna pod adresem: `day_by_day_reporting`

Wskazówki:
 - `Wizualizacja danych > Dzień 4 > Callback context`,
 - `Wizualizacja danych > Dzień 4 > Aplikacja multipage`.
 - Używając komponentu [html.Button](https://community.plotly.com/t/button-with-link/11809), możesz łatwo stworzyć ładniejsze linki do poszczególnych stron.

 Tutaj przygotuj layout aplikacji

In [116]:
app = dash.Dash(__name__)
app.layout = html.Div([
  dcc.Location(id='url', refresh=False),
  html.Button(dcc.Link("TOP Report", href=f"url")),
  html.Button(dcc.Link("Comparison", href=f"/second_page")),
  html.Button(dcc.Link("Day By Day Reporting", href=f"/third_page")),
  html.Br(),
  html.Div(id='page-content')
])
layout_page_one = top_routes_layout
layout_page_two = comparision_layout
layout_page_three = day_by_day_layout

 Tutaj stwórz procedurę odpowiadającą za nawigację pomiędzy poszczególnymi stronami

In [117]:
@app.callback(
  Output('page-content', 'children'),
  Input('url', 'pathname')
)
def display_page_content(pathname):
  if pathname == "/second_page":
    return layout_page_two
  elif pathname == "/third_page":
    return layout_page_three
  else:
    return layout_page_one

 Tutaj uruchom aplikację

In [118]:
thread = threading.Thread(target=app.run_server)
thread.start()

 # Podsumowanie
 W tym notatniku stworzyliśmy zgodnie z wymaganiami raport do dyspozycji osób decyzyjnych. Kod źródłowy może w tym momencie zostać przekazany dalej, do działu IT, który następnie wdroży rozwiązanie na serwer dostępny dla każdej zainteresowanej osoby. W praktyce oznacza to koniec naszej pracy nad tym zadaniem. Choć warto dodać, że często po udostępnieniu raportu pojawiają się dodatkowe wymagania oraz komentarze ze względu na informacje, które są tam zawarte.

 W kolejnym notatniku podsumujemy sobie cały warsztat.